# Seq2Seq模型实现文本翻译

## 概述


In [1]:
# 训练、验证、测试数据集下载地址
urls = {
    'train': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/training.tar.gz',
    'valid': 'http://www.quest.dcs.shef.ac.uk/wmt16_files_mmt/validation.tar.gz',
    'test': 'http://www.quest.dcs.shef.ac.uk/wmt17_files_mmt/mmt_task1_test2016.tar.gz'
}

## 数据准备

### 词典

在文本预处理中，**词典（vocabulary）** 将词元（token）映射到从0开始的数字索引（index）中。

词元，即文本的基本单元，可以是单词（word）或者字符（char）。本篇教程选择单词作为词元，并在构建词典中忽略出现次数少于`min_freq`（默认为1）的词元。

In [2]:
class Vocab:
    def __init__(self, word_count_dict, min_freq=1, special_tokens=['<unk>', '<pad>', '<bos>', '<eos>']):
        self.word2idx = {}
        for idx, tok in enumerate(special_tokens):
            self.word2idx[tok] = idx

        # 过滤低词频的词元
        filted_dict = {w: c for w, c in word_count_dict.items() if c >= min_freq}
        for w, _ in filted_dict.items():
            self.word2idx[w] = len(self.word2idx)
        
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        self.bos_idx = self.word2idx['<bos>'] # 特殊占位符：句首
        self.eos_idx = self.word2idx['<eos>'] # 特殊占位符：句末
        self.pad_idx = self.word2idx['<pad>'] # 特殊占位符：补充字符
        self.unk_idx = self.word2idx['<unk>'] # 特殊占位符：低词频词元或未曾出现的词元

    def _word2idx(self, word):
        # 单词映射至数字索引
        if word not in self.word2idx:
            return self.unk_idx
        return self.word2idx[word]
    
    def _idx2word(self, idx):
        # 数字索引映射至单词
        if idx not in self.idx2word:
            raise ValueError('input index is not in vocabulary.')
        return self.idx2word[idx]
    
    def encode(self, word_or_list):
        # 将单个单词或单词数组映射至单个数字索引或数字索引数组
        if isinstance(word_or_list, list):
            return [self._word2idx(i) for i in word_or_list]
        return self._word2idx(word_or_list)
    
    def decode(self, idx_or_list):
        # 将单个数字索引或数字索引数组映射至单个单词或单词数组
        if isinstance(idx_or_list, list):
            return [self._idx2word(i) for i in idx_or_list]
        return self._idx2word(idx_or_list)
    
    def __len__(self):
        return len(self.word2idx)

通过自定义词频字典进行测试，我们可以看到词典已去除词频少于2的词元c，并加入了默认的四个特殊占位符，故词典整体长度为：4 - 1 + 4 = 7

In [3]:
word_count = {'a':20, 'b':10, 'c':1, 'd':2}

vocab = Vocab(word_count, min_freq=2)
len(vocab)

7

### Multi30K数据集

**Multi30K**是一个大规模的图像-文本数据集，包含30K+图片，每张图片对应两类不同的文本描述：
- 英语描述，及对应的德语翻译；
- 五个独立的、非翻译而来的英语和德语描述，描述中包含的细节并不相同；

因其收集的不同语言对于图片的描述相互独立，所以训练出的模型可以更好地适用于有噪声的多模态内容。

![avatar](./assets/Multi30K.png)

### 创建数据加载器

Multi30K的训练、验证、测试数据集各对应一个`tar.gz`文件，其中包含en和de两部分。我们使用Python的`tarfile`对数据进行读取，并对文本进行统一大小写，去除空格，分词等预处理。以训练数据集为例，原始的解压目录如下：
```text
training
├── train.de
└── train.en
```

我们将使用`spaCy`创建分词器（tokenizer），将de、en文件中的文本从一句话转化为组成该句子的单个词元列表。

> 在进行分词前，需手动安装`spaCy`及对应语言的模型，代码如下：

> pip install spacy

> python -m spacy download de_core_news_sm

> python -m spacy download en_core_web_sm

In [10]:
import re
import six
import string
import tarfile
import spacy
from functools import partial

class Multi30K():
    """Multi30K数据集加载器
    
    加载Multi30K数据集并处理为一个Python迭代对象。
    
    """
    def __init__(self, path):
        self.data = self._load(path)
        
    def _load(self, path):
        def tokenize(text, spacy_lang):
            # 去除多余空格，统一大小写
            text = text.rstrip()
            return [tok.text.lower() for tok in spacy_lang.tokenizer(text)]
        
        # 加载英、德语分词器
        tokenize_de = partial(tokenize, spacy_lang=spacy.load('de_core_news_sm'))
        tokenize_en = partial(tokenize, spacy_lang=spacy.load('en_core_web_sm'))
        
        # 读取Multi30K数据，并进行分词
        tarf = tarfile.open(path)
        members = {i.name.split('.')[-1]: i for i in tarf.getmembers()}
        de = tarf.extractfile(members['de']).readlines()[:-1]
        de = [tokenize_de(i.decode()) for i in de]
        en = tarf.extractfile(members['en']).readlines()[:-1]
        en = [tokenize_en(i.decode()) for i in en]

        return list(zip(de, en))
        
    def __getitem__(self, idx):
        return self.data[idx]
    
    def __len__(self):
        return len(self.data)

### 创建数据迭代器

In [5]:
import mindspore

class Iterator():
    def __init__(self, dataset, de_vocab, en_vocab, batch_size, max_len=32, drop_reminder=False):
        self.dataset = dataset
        self.de_vocab = de_vocab
        self.en_vocab = en_vocab
        
        self.batch_size = batch_size
        self.max_len = max_len
        self.drop_reminder = drop_reminder

        length = len(self.dataset) // batch_size 
        self.len = length if drop_reminder else length + 1
    
    def __call__(self):
        def pad(idx_list, vocab, max_len):
            idx_pad_list, idx_len = [], []
            # max_len = max([len(i) for i in idx_list]) + 2
            for i in idx_list:
                if len(i) > max_len - 2:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i[:max_len-2] + [vocab.eos_idx]
                    )
                    idx_len.append(max_len)
                else:
                    idx_pad_list.append(
                        [vocab.bos_idx] + i + [vocab.eos_idx] + [vocab.pad_idx] * (max_len - len(i) - 2)
                    )
                    idx_len.append(len(i) + 2)
            return idx_pad_list, idx_len

        def sort_by_length(src, trg):
            data = zip(src, trg)
            data = sorted(data, key=lambda t: len(t[0]), reverse=True)
            return zip(*list(data))
            
        def encode_and_pad(batch_data, max_len):
            src_data, trg_data = zip(*batch_data)
            src_idx = [self.de_vocab.encode(i) for i in src_data]
            trg_idx = [self.en_vocab.encode(i) for i in trg_data]
            
            src_idx, trg_idx = sort_by_length(src_idx, trg_idx)
            src_idx_pad, src_len = pad(src_idx, de_vocab, max_len)
            trg_idx_pad, _ = pad(trg_idx, en_vocab, max_len)
            
            return src_idx_pad, src_len, trg_idx_pad
        
        for i in range(self.len):
            if i == self.len - 1 and not self.drop_reminder:
                batch_data = self.dataset[i * self.batch_size:]
            else:
                batch_data = self.dataset[i * self.batch_size: (i+1) * self.batch_size]
            
            src_idx, src_len, trg_idx = encode_and_pad(batch_data, self.max_len)
            yield mindspore.Tensor(src_idx, mindspore.int32), \
                mindspore.Tensor(src_len, mindspore.int32), \
                mindspore.Tensor(trg_idx, mindspore.int32)
    
    def __len__(self):
        return self.len

### 数据下载模块

数据下载模块使用`requests`库进行http请求，并通过`tqdm`对下载百分比进行可视化。此外针对下载安全性，使用IO的方式下载临时文件，而后保存至指定的路径并返回。
> `tqdm`和`requests`库需手动安装，命令如下：`pip install tqdm requests`

In [6]:
import os
import logging
import shutil
import requests
import tempfile
from tqdm import tqdm
from typing import IO
from pathlib import Path

# 指定保存路径为 `home_path/.mindspore_examples`
cache_dir = Path.home() / '.mindspore_examples'

def http_get(url: str, temp_file:IO):
    """使用requests库下载数据，并使用tqdm库进行流程可视化"""
    req = requests.get(url, stream=True)
    content_length = req.headers.get('Content-Length')
    total = int(content_length) if content_length is not None else None
    progress = tqdm(unit='B', total=total)
    for chunk in req.iter_content(chunk_size=1024):
        if chunk:
            progress.update(len(chunk))
            temp_file.write(chunk)
    progress.close()

def download(file_name:str, url: str):
    """下载数据并存为指定名称"""
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir)
    cache_path = os.path.join(cache_dir, file_name)
    cache_exist = os.path.exists(cache_path)
    if not cache_exist:
        with tempfile.NamedTemporaryFile() as temp_file:
            http_get(url, temp_file)
            temp_file.flush()
            temp_file.seek(0)
            logging.info(f"copying {temp_file.name} to cache at {cache_path}")
            with open(cache_path, 'wb') as cache_file:
                shutil.copyfileobj(temp_file, cache_file)
    return cache_path

下载数据集，并加载数据。

In [32]:
def download_dataset(urls):
    train_path = download('train.tar.gz', urls['train'])
    valid_path = download('valid.tar.gz', urls['valid'])
    test_path = download('test.tar.gz', urls['test'])
    
    return Multi30K(train_path), Multi30K(valid_path), Multi30K(test_path)

In [33]:
train_dataset, valid_dataset, test_dataset = download_dataset(urls)

完成Multi30K数据集加载后，对加载好的数据集进行测试，打印测试数据集第一组英德语描述的分词结果：

In [34]:
for de, en in test_dataset:
    print(de, en)
    break

['ein', 'mann', 'mit', 'einem', 'orangefarbenen', 'hut', ',', 'der', 'etwas', 'anstarrt', '.'] ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']


### 词典构建

将文本数据进行初步处理并分词后，通过`collections`中的`Counter`和`OrderedDict`统计英/德语每个单词在整体文本中出现的频率。构建词频字典的目的是在构建词典时，通过词频对单词进行排序，保证常用的单词对应数值较小的索引，并且通过词频，排除掉出现频率过低的单词，节约储存空间。

In [35]:
from collections import Counter, OrderedDict

def build_vocab(dataset):
    de_words, en_words = [], []
    for de, en in dataset:
        de_words.extend(de)
        en_words.extend(en)
        
    de_count_dict = OrderedDict(sorted(Counter(de_words).items(), key=lambda t: t[1], reverse=True))
    en_count_dict = OrderedDict(sorted(Counter(en_words).items(), key=lambda t: t[1], reverse=True))
    
    return Vocab(de_count_dict, min_freq=2), Vocab(en_count_dict, min_freq=2)

In [36]:
de_vocab, en_vocab = build_vocab(train_dataset)
len(de_vocab)

7853

## 模型构建

### Encoder

In [11]:
import mindspore
import mindspore.nn as nn
import mindspore.ops as ops
import mindspore.numpy as mnp

class Encoder(nn.Cell):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True).to_float(compute_dtype)
        self.fc = nn.Dense(enc_hid_dim * 2, dec_hid_dim).to_float(compute_dtype)

        self.dropout = nn.Dropout(1-dropout)
        
    def construct(self, src, src_len):
        #src = [src len, batch size]
        #src_len = [batch size]
        embedded = self.dropout(self.embedding(src))
        #embedded = [src len, batch size, emb dim]
                        
        outputs, hidden = self.rnn(embedded, seq_length=src_len)
                                 
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = ops.tanh(self.fc(mnp.concatenate((hidden[-2,:,:], hidden[-1,:,:]), axis = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        # print(hidden)
        return outputs, hidden

### Attention

In [12]:
class Attention(nn.Cell):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Dense((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim).to_float(compute_dtype)
        self.v = nn.Dense(dec_hid_dim, 1, has_bias = False).to_float(compute_dtype)
        
    def construct(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = mnp.tile(hidden.expand_dims(1), (1, src_len, 1))
  
        encoder_outputs = encoder_outputs.transpose(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = ops.tanh(self.attn(mnp.concatenate((hidden, encoder_outputs), axis = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return ops.Softmax(1)(attention)

### Decoder

In [13]:
class Decoder(nn.Cell):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim).to_float(compute_dtype)
        self.fc_out = nn.Dense((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim).to_float(compute_dtype)
        self.dropout = nn.Dropout(1-dropout)
        
    def construct(self, inputs, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        inputs = inputs.expand_dims(0)
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(inputs))
        
        #embedded = [1, batch size, emb dim]
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.expand_dims(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.transpose(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = ops.BatchMatMul()(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.transpose(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = mnp.concatenate((embedded, weighted), axis = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.expand_dims(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(mnp.concatenate((output, weighted, embedded), axis = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)

### Seq2Seq

In [ ]:
import random

class Seq2Seq(nn.Cell):
    def __init__(self, encoder, decoder, src_pad_idx, teacher_forcing_ration):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.teacher_forcing_ratio = teacher_forcing_ration
        self.random = ops.UniformReal()
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).astype(mindspore.int32).swapaxes(1, 0)
        return mask
        
    def construct(self, src, src_len, trg, trg_len=None):
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        if trg_len is None:
            trg_len = trg.shape[0]
        
        #tensor to store decoder outputs
        outputs = []
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        inputs = trg[0]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(inputs, hidden, encoder_outputs, mask)
            # print(output)
            #place predictions in a tensor holding predictions for each token
            outputs.append(output)
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 

            if self.training:
                #decide if we are going to use teacher forcing or not
                teacher_force = self.random((1,)) < self.teacher_forcing_ratio
                # teacher_force = random.random() < self.teacher_forcing_ratio
                #if teacher forcing, use actual next token as next input
                #if not, use predicted token
                inputs = trg[t] if teacher_force else top1
            else:
                inputs = top1
        
        outputs = mnp.stack(outputs, axis=0)
            
        return outputs.astype(dtype)

In [41]:
import mindspore as ms
import numpy as np
from mindspore.nn.layer.rnn_cells import _rnn_tanh_cell
from mindspore._checkparam import Validator as validator
from mindspore.ops import Squeeze, Stack

input_x = ms.Tensor(np.ones(shape=[3, 2, 1]))
squeeze = Squeeze(2)
output_x = squeeze(input_x)

output_x

Tensor(shape=[3, 2], dtype=Float64, value=
[[1.00000000e+000, 1.00000000e+000],
 [1.00000000e+000, 1.00000000e+000],
 [1.00000000e+000, 1.00000000e+000]])

In [51]:
input_array = np.array([[[1],[2]],[[3],[4]],[[5],[6]]])
input_x = ms.Tensor(input_array)
test_x = input_x[2:]
test_x

Tensor(shape=[0, 2, 1], dtype=Int32, value=
)

In [44]:
input_x = ms.Tensor(np.ones(shape=[3, 2, 1]))
input_x

Tensor(shape=[3, 2, 1], dtype=Float64, value=
[[[1.00000000e+000],
  [1.00000000e+000]],
 [[1.00000000e+000],
  [1.00000000e+000]],
 [[1.00000000e+000],
  [1.00000000e+000]]])

In [57]:
input_x = ms.Tensor(np.ones(shape=[5, 2, 28]))
output_x = input_x[3:4:1]
squeeze = Squeeze(0)
output_x = squeeze(output_x)
output_x

Tensor(shape=[2, 28], dtype=Float64, value=
[[1.00000000e+000, 1.00000000e+000, 1.00000000e+000 ... 1.00000000e+000, 1.00000000e+000, 1.00000000e+000],
 [1.00000000e+000, 1.00000000e+000, 1.00000000e+000 ... 1.00000000e+000, 1.00000000e+000, 1.00000000e+000]])